In [31]:
# import libraries
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians
import os.path


# define functions
%run distanceMatrix.py
# instance size
N=5
# number of trucks
K=10
# maximum tour time
T=int(10 * 60) # 4.5h = 270min

# load the data and files
clientsFile = 'clients'+str(N)+'.csv'
# 0     =7am
# 600   =5pm
# noon  =300
# 1pm   =360
# 4pm   =540
clients = pd.read_csv(clientsFile)
wpfsFile = 'WPF.csv'
citiesFile = 'belgian-cities-geocoded.csv'
cities = pd.read_csv(citiesFile)
distanceMatrixFile = 'distanceMatrix'+str(N)+'.csv'


# get the matrix and save it if necessary
if os.path.isfile(distanceMatrixFile):
    print('exists')
    distanceMatrix = pd.read_csv(distanceMatrixFile).to_numpy()
else:
    print('does not exist')
    distanceMatrix = createDistanceMatrix(clientsFile, citiesFile, wpfsFile)
    pd.DataFrame(distanceMatrix).to_csv(distanceMatrixFile, index=False) 

# get coordinates 
coords = [(50.9338827,4.5605498)] # depot
for client in clients['Place']:
    idx = cities.index[cities['name']==client].tolist()[0]
    coords.append((cities.iloc[idx]['lat'],cities.iloc[idx]['lng']))

# get timewindows
timeWindows=[(0,600)] #depot
for id in clients['ClientID']:
    opening=clients['opening'][id-1]
    closing=clients['closing'][id-1]
    opening=int(opening)
    closing=int(closing)
    timeWindow=(opening,closing)
    timeWindows.append(timeWindow)

does not exist


In [13]:
# build the model and solve it
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # data['time_matrix'] = [
    #     [0,7.07,7.07,7.07,7.07],
    #     [7.07,0,10,2*7.07,10],
    #     [7.07,10,0,10,2*7.07],
    #     [7.07,2*7.07,10,0,10],
    #     [7.07,10,2*7.07,10,0]
    # ]
    data['time_matrix'] = distanceMatrix
    data['time_windows'] = timeWindows
    # data['time_windows'] = [
    #     (0, 600),  # depot
    #     (0, 600),  # 1
    #     (0, 600),  # 2
    #     (0, 600),  # 3
    #     (0, 600),  # 4
    #     (0, 600)
    # ]
    data['num_vehicles'] = K
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))

def get_cumul_data(solution, routing, dimension):
  """Get cumulative data from a dimension and store it in an array."""
  # Returns an array cumul_data whose i,j entry contains the minimum and
  # maximum of CumulVar for the dimension at the jth node on route :
  # - cumul_data[i][j][0] is the minimum.
  # - cumul_data[i][j][1] is the maximum.

  cumul_data = []
  for route_nbr in range(routing.vehicles()):
    route_data = []
    index = routing.Start(route_nbr)
    dim_var = dimension.CumulVar(index)
    route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      dim_var = dimension.CumulVar(index)
      route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    cumul_data.append(route_data)
  return cumul_data

def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

########################################
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

# Create and register a transit callback.
def time_callback(from_index, to_index):
    """Returns the travel time between the two nodes."""
    # Convert from routing variable Index to time matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['time_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(time_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Time Windows constraint.
time = 'Time'
routing.AddDimension(
    transit_callback_index,
    30,  # allow waiting time
    T,  # maximum time per vehicle
    False,  # Don't force start cumul to zero.
    time)
time_dimension = routing.GetDimensionOrDie(time)
# Add time window constraints for each location except depot.
for location_idx, time_window in enumerate(data['time_windows']):
    if location_idx == data['depot']:
        continue
    index = manager.NodeToIndex(location_idx)
    time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
# Add time window constraints for each vehicle start node.
depot_idx = data['depot']
for vehicle_id in range(data['num_vehicles']):
    index = routing.Start(vehicle_id)
    time_dimension.CumulVar(index).SetRange(
        data['time_windows'][depot_idx][0],
        data['time_windows'][depot_idx][1])

# Instantiate route start and end times to produce feasible times.
for i in range(data['num_vehicles']):
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.Start(i)))
    routing.AddVariableMinimizedByFinalizer(
        time_dimension.CumulVar(routing.End(i)))

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
# maximum computation time
search_parameters.time_limit.seconds = 30

# search_parameters = pywrapcp.DefaultRoutingSearchParameters()
# search_parameters.local_search_metaheuristic = (
#     routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)
# search_parameters.time_limit.seconds = 60
# search_parameters.log_search = True


# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)
    # get solution windows
    solution_windows=get_cumul_data(solution,routing,time_dimension)
    solution_routes=get_routes(solution,routing,manager)
    
else:
    print("INFEASIBLE or RAN OUT OF COMPUTATION TIME")  

Objective: 451
Route for vehicle 0:
0 Time(0,0) -> 4 Time(45,45) -> 2 Time(125,125) -> 0 Time(140,140)
Time of the route: 140min

Route for vehicle 1:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 2:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 3:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 4:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 5:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 6:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 7:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 8:
0 Time(0,0) -> 0 Time(0,0)
Time of the route: 0min

Route for vehicle 9:
0 Time(0,0) -> 5 Time(41,41) -> 3 Time(139,139) -> 1 Time(271,271) -> 0 Time(311,311)
Time of the route: 311min

Total time of all routes: 451min


In [14]:
# Map the solution
import folium

# list of colors to be used in the map
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred','black', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray','red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# zoom in on Belgium
map = folium.Map(location=[51,5], zoom_start = 8)

# visualize the routes
solution_routes_nonEmpty = [route for route in solution_routes if len(route) > 2]
for idx,route in enumerate(solution_routes_nonEmpty):
    route_coords=[]
    for node in route:
        route_coords.append(coords[node])
    folium.PolyLine(route_coords,weight=len(route),color=colors[idx],popup=route).add_to(map)

# visualize the nodes
for idx,coord in enumerate(coords):
    if(idx == 0):
        folium.Marker(
            location=[coord[0], coord[1]],
            popup='Kampenhout-0',
            icon=folium.Icon(color='black',icon="home"),
        ).add_to(map)
    else:
        icon_colors = ['green','lightblue','cadetblue','orange','lightgray','red']
        folium.Marker(
            location=[coord[0], coord[1]],
            popup=clients['Place'][idx-1]+'-'+str(clients['ActionType'][idx-1]),
            icon=folium.Icon(color=icon_colors[clients['ActionType'][idx-1]-1], icon='', prefix='fa')
        ).add_to(map)

map

In [29]:
a=0
b=4
distanceKmFromCoord(
    coords[a][0],coords[a][1],
    coords[b][0],coords[b][1] 
)+6+25

42.768791180718544